In [ ]:
import numpy as np
import torch
table = torch.zeros((3, 3, 3))
feature = torch.tensor([1, 2]) # old_state
q_values = table[*feature]
print(q_values)

In [ ]:
import torch
PATH = "model/my-model_stats.pt"
table = torch.load(PATH)
print(table.shape)
print(torch.sum(table > 0))

In [16]:
%cd "/home/gromovl/Programming/ml_final-project/mle-ss23_final-project"

/home/gromovl/Programming/ml_final-project/mle-ss23_final-project


In [17]:
import numpy as np
from helper import get_safety_feature, get_allowed_actions, get_blast_coords
field = np.full((17, 17), -1) # empty field

# Place walls in between
aisles = {} #store locations in a dictionary
for i in range(1,8):
   aisles[2*i-1] = [i for i in range(1,16)] #3,5,7,9,11,13,15
   aisles[2*i] = [1,3,5,7,9,11,13,15] #2,4,6,8,10,12,14,16

aisles[15] = [i for i in range(1,16)]


# Set all walls
for row_index in range(1,16):
  for column_index in aisles[row_index]:
    field[row_index, column_index] = 0

#print(np.sum(field == 0))


my_x, my_y = agent_pos = (7, 9)

explosion_map = np.zeros((17, 17))
explosion_map[my_x - 1, my_y] = 1

bombs = [((agent_pos[0] + 3, agent_pos[1]), 0), 
         #((agent_pos[0], agent_pos[1] - 2), 1)
        ]

crates = np.array([#(agent_pos[0], agent_pos[1] - 1),
                   #(agent_pos[0], agent_pos[1] + 1),
                   #(agent_pos[0] - 1, agent_pos[1]),
                  ])

if len(crates) > 0:
  field[tuple(crates.T)] = 2

print("UP", "RIGHT", "DOWN", "LEFT", "WAIT")
print("Safety feature", get_safety_feature(agent_pos, field, explosion_map, bombs))

field[agent_pos] = 2

field[explosion_map > 0] = 4

for bomb in bombs:
  b = bomb[0]
  field[b] = 6

print(field.T)


UP RIGHT DOWN LEFT WAIT
Safety feature tensor([1, 0, 1, 0, 0])
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  4  2  0  0  6  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [18]:
import torch
table_path = "agent_code/qtable_agent/model/my-model.pt"
table = torch.load(table_path)
print(f"Table shape {table.shape}")
print(f"Number of elements {table.numel()}")
print(f"RAM {table.numel() * table.element_size() / (2**20)} MB")

Table shape torch.Size([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6])
Number of elements 98304
RAM 0.75 MB


In [2]:
import torch
table_stat_path = "model/my-model_stats.pt"
table_stats = torch.load(table_stat_path)
print(f"Explored state ratio: {torch.sum(table_stats > 0) / table_stats.numel():.2%}")

Explored state ratio: 0.82%


In [19]:
from matplotlib import pyplot as plt
import numpy as np
state_action_table = table_stats.reshape(-1, 6)
s = np.arange(0, state_action_table.shape[0])
a = np.arange(0, state_action_table.shape[1])
S, A = np.meshgrid(a, s)
plt.contourf(S, A, state_action_table)
plt.colorbar()

plt.show()

: 

In [13]:
%cd "/home/gromovl/Programming/ml_final-project/mle-ss23_final-project"
# TODO : matplotlib seems slow?
import pstats
from pstats import SortKey
p = pstats.Stats("profile.log")
p.sort_stats(SortKey.TIME).print_stats(10)
p.print_callers(10)

/home/gromovl/Programming/ml_final-project/mle-ss23_final-project
Tue Sep 12 19:39:03 2023    profile.log

         6064834 function calls (5936837 primitive calls) in 27.240 seconds

   Ordered by: internal time
   List reduced from 13031 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    1.377    0.138    1.379    0.138 {built-in method show}
  169/166    1.340    0.008    1.365    0.008 {built-in method _imp.create_dynamic}
     2401    0.656    0.000    0.656    0.000 {method 'read' of '_io.BufferedReader' objects}
432738/432302    0.618    0.000    0.751    0.000 {built-in method builtins.isinstance}
     2372    0.475    0.000    0.475    0.000 {built-in method marshal.loads}
239657/239382    0.432    0.000    0.727    0.000 {built-in method builtins.getattr}
     6798    0.399    0.000    0.763    0.000 /home/gromovl/miniconda3/envs/ml_final-project/lib/python3.11/inspect.py:867(cleandoc)
111359/91542    0.369    0